In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import streamlit as st
import keras
from keras.layers import Dropout,BatchNormalization,LSTM,Bidirectional,GlobalMaxPool1D,Input,Activation,Flatten,Embedding,Dense,concatenate,Conv1D,MaxPooling1D
import string
import re
from tqdm import tqdm
import nltk
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
import spacy
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,f1_score,recall_score,precision_score,classification_report
import os
from keras.models import Model
import seaborn as sns
from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import warnings
warnings.filterwarnings('ignore')
import tensorboard
from textblob import TextBlob
import os
import tensorboard
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
import datetime
from keras.initializers import he_normal
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from prettytable import PrettyTable
from better_profanity import profanity
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from tensorflow.keras.models import load_model
import pickle

In [2]:
data=pd.read_csv("train-balanced-sarcasm.csv")

In [3]:
data['parent_comment'][44]

"Clinton campaign accuses FBI of 'blatant double standard'"

In [4]:
data['comment'][44]

'wow it is totally unreasonable to assume that the agency that covered up Bush war crimes because "muh republican party" would be partisan as fuck'

In [5]:
# slangs
strings='''AFAIK=As Far As I Know
AFK=Away From Keyboard
ASAP=As Soon As Possible
ATK=At The Keyboard
ATM=At The Moment
A3=Anytime, Anywhere, Anyplace
BAK=Back At Keyboard
BBL=Be Back Later
BBS=Be Back Soon
BFN=Bye For Now
B4N=Bye For Now
BRB=Be Right Back
BRT=Be Right There
BTW=By The Way
B4=Before
B4N=Bye For Now
CU=See You
CUL8R=See You Later
CYA=See You
FAQ=Frequently Asked Questions
FC=Fingers Crossed
FWIW=For What It's Worth
FYI=For Your Information
GAL=Get A Life
GG=Good Game
GN=Good Night
GMTA=Great Minds Think Alike
GR8=Great!
G9=Genius
IC=I See
ICQ=I Seek you (also a chat program)
ILU=ILU: I Love You
IMHO=In My Honest/Humble Opinion
IMO=In My Opinion
IOW=In Other Words
IRL=In Real Life
KISS=Keep It Simple, Stupid
LDR=Long Distance Relationship
LMAO=Laugh My A.. Off
LOL=Laughing Out Loud
LTNS=Long Time No See
L8R=Later
MTE=My Thoughts Exactly
M8=Mate
NRN=No Reply Necessary
OIC=Oh I See
PITA=Pain In The A..
PRT=Party
PRW=Parents Are Watching
QPSA?=Que Pasa?
ROFL=Rolling On The Floor Laughing
ROFLOL=Rolling On The Floor Laughing Out Loud
ROTFLMAO=Rolling On The Floor Laughing My A.. Off
SK8=Skate
STATS=Your sex and age
ASL=Age, Sex, Location
THX=Thank You
TTFN=Ta-Ta For Now!
TTYL=Talk To You Later
U=You
U2=You Too
U4E=Yours For Ever
WB=Welcome Back
WTF=What The F...
WTG=Way To Go!
WUF=Where Are You From?
W8=Wait...
7K=Sick:-D Laugher'''

In [6]:
# with open('dictionary_values.pkl','rb') as file1:
#             dictionary_values=pickle.load(file1)
def modeling():
    model=load_model(r'C:\Users\DELL\Desktop\Sarcasm\parameter_files\model_cnn123.h5')
    with open('dictionary_values.pkl','rb') as file1:
            dictionary_values=pickle.load(file1)
   
    return model,dictionary_values

In [7]:
model,dictionary_values=modeling()

In [8]:
dictionary_values

{'AFAIK': 'As Far As I Know',
 'AFK': 'Away From Keyboard',
 'ASAP': 'As Soon As Possible',
 'ATK': 'At The Keyboard',
 'ATM': 'At The Moment',
 'A3': 'Anytime, Anywhere, Anyplace',
 'BAK': 'Back At Keyboard',
 'BBL': 'Be Back Later',
 'BBS': 'Be Back Soon',
 'BFN': 'Bye For Now',
 'B4N': 'Bye For Now',
 'BRB': 'Be Right Back',
 'BRT': 'Be Right There',
 'BTW': 'By The Way',
 'B4': 'Before',
 'CU': 'See You',
 'CUL8R': 'See You Later',
 'CYA': 'See You',
 'FAQ': 'Frequently Asked Questions',
 'FC': 'Fingers Crossed',
 'FWIW': "For What It's Worth",
 'FYI': 'For Your Information',
 'GAL': 'Get A Life',
 'GG': 'Good Game',
 'GN': 'Good Night',
 'GMTA': 'Great Minds Think Alike',
 'GR8': 'Great!',
 'G9': 'Genius',
 'IC': 'I See',
 'ICQ': 'I Seek you (also a chat program)',
 'ILU': 'ILU: I Love You',
 'IMHO': 'In My Honest/Humble Opinion',
 'IMO': 'In My Opinion',
 'IOW': 'In Other Words',
 'IRL': 'In Real Life',
 'KISS': 'Keep It Simple, Stupid',
 'LDR': 'Long Distance Relationship',
 'LM

In [9]:
# x1=strings.split("\n")
# dict1={}
# for i in x1:
#     x2=(i.split("="))
#     dict1[x2[0]]=x2[1]
# print(dict1)

In [10]:
# removing stopwords

# we have removed stopwords like no,not,nor.
stop_words=["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", 
            "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", 
            "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", 
            "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being",
            "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but",
            "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", 
            "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", 
            "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", 
            "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few",
            "more", "most", "other", "some", "such" "only", "own", "same", "so", "than", "too", "very", 
            "s", "t", "can", "will", "just", "don", "should", "now"]

In [11]:
# path_to_glove_file = r'C:\Users\DELL\Desktop\Sarcasm\glove.6B.100d.txt'

# # https://keras.io/examples/nlp/pretrained_word_embeddings/

# embeddings_index = {}
# with open(path_to_glove_file,encoding='utf-8') as f:
#     for line in f:
#         word, coefs = line.split(maxsplit=1)
#         coefs = np.fromstring(coefs, "f", sep=" ")
#         embeddings_index[word] = coefs
        
        
# print("Found %s word vectors." % len(embeddings_index))

In [12]:
def chat(text):
    new_text=[]
    for word in text.split():
        if word.upper() in dictionary_values:
            new_text.append(dictionary_values[word.upper()])
        else:
            new_text.append(word)
            
    done=" ".join(new_text)

            
    return done


#decontract words
def decontracted(phrase):
    
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    
    return phrase


def stopwords1(text):
    new_list=[]
    for word in text.split():
        if word in stop_words:
            new_list.append("")
        else:
            new_list.append(word)

    done=list(filter(None,new_list))
    done=" ".join(done)
    
    return done


# remove html tags
def remove_html(text):
    return re.sub(r'<.*?>',"",text)


# removing digits
def remove_numbers(text):
    return re.sub("\d+", "", text)



string1=string.punctuation
string1=list(string1)
string1.remove('!')
string1.remove('?')
print(string1)


def remove_punctuation(data):
    for char in string1:
        if char in data:
            data=data.replace(char," ")
    return data

['"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']


In [13]:
def preprocess_text(text):
    pre_text=[]
    text=chat(text)
    text=decontracted(text)
    text=text.lower()
    text=stopwords1(text)
    text=remove_html(text)
    text=remove_numbers(text)
    text=remove_punctuation(text)
    
    pre_text.append(text)
    
    return pre_text

In [14]:
def profanity_words(text):
    list1=[]
    for sentence in (text):
        profane_word=profanity.contains_profanity(sentence)
        list1.append(profane_word)
        
    return list1


def sentiment_subjectivity(text):
    list1=[]
    for sentence in (text):
        subjectivity=TextBlob(sentence).sentiment.subjectivity
        list1.append(subjectivity)
    return list1


def sentiment_intensity(text):
    neg_list,pos_list,neutral_list=[],[],[]
    for sentence in (text):
        sentiment_object= SentimentIntensityAnalyzer()
        polarity_scores=sentiment_object.polarity_scores(sentence)
        
        neg_list.append(polarity_scores['neg'])
        pos_list.append(polarity_scores['pos'])
        neutral_list.append(polarity_scores['neu'])
        
    return neg_list,pos_list,neutral_list
        
    
## Exclamation mark

def count_exclamation(text):
    list1=[]
    for i in text:
        if '!' in i:
            list1.append(1)
        else:
            list1.append(0)
            
    return list1



## question mark

def count_question(text):
    list1=[]
    for i in text:
        if '?' in i:
            list1.append(1)
        else:
            list1.append(0)
            
    return list1

In [15]:
# tokenizer=Tokenizer(num_words=40000,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',oov_token='<OOV>')
# tokenizer.fit_on_texts(list1)
# x=tokenizer.texts_to_sequences(list1)
# maxlen = 100
# print(maxlen)
# word_index=tokenizer.word_index
# print(len(word_index)+1)

# padded_sequences=pad_sequences(x,maxlen=maxlen,padding='post',truncating='post')

In [16]:
def profanity_words(text):
    list1=[]
    for sentence in (text):
        profane_word=profanity.contains_profanity(sentence)
        list1.append(profane_word)
        
    return list1


def sentiment_subjectivity(text):
    list1=[]
    for sentence in (text):
        subjectivity=TextBlob(sentence).sentiment.subjectivity
        list1.append(subjectivity)
    return list1


def sentiment_intensity(text):
    neg_list,pos_list,neutral_list=[],[],[]
    for sentence in (text):
        sentiment_object= SentimentIntensityAnalyzer()
        polarity_scores=sentiment_object.polarity_scores(sentence)
        
        neg_list.append(polarity_scores['neg'])
        pos_list.append(polarity_scores['pos'])
        neutral_list.append(polarity_scores['neu'])
        
    return neg_list,pos_list,neutral_list
        
    
## Exclamation mark

def count_exclamation(text):
    list1=[]
    for i in text:
        if '!' in i:
            list1.append(1)
        else:
            list1.append(0)
            
    return list1



## question mark

def count_question(text):
    list1=[]
    for i in text:
        if '?' in i:
            list1.append(1)
        else:
            list1.append(0)
            
    return list1

In [17]:
# def final_func1(text):
#     preprocess_list=preprocess_text(text)
#     tokenizer=Tokenizer(num_words=40000,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',oov_token='<OOV>')
#     tokenizer.fit_on_texts(preprocess_list)
#     x=tokenizer.texts_to_sequences(preprocess_list)
#     maxlen = 100
#     word_index=tokenizer.word_index

#     padded_sequences=pad_sequences(x,maxlen=maxlen,padding='post',truncating='post')
    
#     sentiment_subj=sentiment_subjectivity(preprocess_list)
#     sentiment_neg,sentiment_pos,sentiment_neu=sentiment_intensity(preprocess_list)
#     exc_mark=count_exclamation(preprocess_list)
#     ques_mark=count_question(preprocess_list)
#     profane_words=profanity_words(preprocess_list)

#     for i in profane_words:

#         profane_words1=[]
#         if i==True:

#             profane_words1.append(1)
#         else :
#             profane_words1.append(0)
    
#     sentiment_subj=np.asarray(sentiment_subj)
#     sentiment_neg=np.asarray(sentiment_neg)
#     sentiment_pos=np.asarray(sentiment_pos)
#     sentiment_neu=np.asarray(sentiment_neu)
#     exc_mark=np.asarray(exc_mark)
#     ques_mark=np.asarray(ques_mark)
#     profane_words1=np.asarray(profane_words1)
#     final=[padded_sequences,sentiment_subj.reshape(-1,1),sentiment_neg.reshape(-1,1),sentiment_pos.reshape(-1,1),sentiment_neu.reshape(-1,1),exc_mark.reshape(-1,1),ques_mark.reshape(-1,1),profane_words1.reshape(-1,1)]
#     pred=model.predict(final)
    
#     return pred
    

In [18]:
def final_func1(text):
    preprocess_list=preprocess_text(text)
    tokenizer=Tokenizer(num_words=40000,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',oov_token='<OOV>')
    tokenizer.fit_on_texts(preprocess_list)
    x=tokenizer.texts_to_sequences(preprocess_list)
    maxlen = 40
    word_index=tokenizer.word_index

    padded_sequences=pad_sequences(x,maxlen=maxlen,padding='post',truncating='post')
    
#     sentiment_subj=sentiment_subjectivity(preprocess_list)
#     sentiment_neg,sentiment_pos,sentiment_neu=sentiment_intensity(preprocess_list)
#     exc_mark=count_exclamation(preprocess_list)
#     ques_mark=count_question(preprocess_list)
#     profane_words=profanity_words(preprocess_list)

#     for i in profane_words:

#         profane_words1=[]
#         if i==True:

#             profane_words1.append(1)
#         else :
#             profane_words1.append(0)
    
#     sentiment_subj=np.asarray(sentiment_subj)
#     sentiment_neg=np.asarray(sentiment_neg)
#     sentiment_pos=np.asarray(sentiment_pos)
#     sentiment_neu=np.asarray(sentiment_neu)
#     exc_mark=np.asarray(exc_mark)
#     ques_mark=np.asarray(ques_mark)
#     profane_words1=np.asarray(profane_words1)
#     final=[padded_sequences,sentiment_subj.reshape(-1,1),sentiment_neg.reshape(-1,1),sentiment_pos.reshape(-1,1),sentiment_neu.reshape(-1,1),exc_mark.reshape(-1,1),ques_mark.reshape(-1,1),profane_words1.reshape(-1,1)]
    final=[padded_sequences]
#     model= load_model('m')
    pred=model.predict(final)
    
    return pred
    

In [19]:
# def main_function(text):
    #     preprocess_list=preprocess_text(text)
    #     tokenizer=Tokenizer(num_words=40000,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',oov_token='<OOV>')
    #     tokenizer.fit_on_texts(preprocess_list)
    #     x=tokenizer.texts_to_sequences(preprocess_list)
    #     maxlen = 100
    #     word_index=tokenizer.word_index

    #     padded_sequences=pad_sequences(x,maxlen=maxlen,padding='post',truncating='post')
        
    #     sentiment_subj=sentiment_subjectivity(preprocess_list)
    #     sentiment_neg,sentiment_pos,sentiment_neu=sentiment_intensity(preprocess_list)
    #     exc_mark=count_exclamation(preprocess_list)
    #     ques_mark=count_question(preprocess_list)
    #     profane_words=profanity_words(preprocess_list)

    #     for i in profane_words:
    #         profane_words1=[]
    #         if i==True:
    #             profane_words1.append(1)
    #         else :
    #             profane_words1.append(0)
        
    #     sentiment_subj=np.asarray(sentiment_subj)
    #     sentiment_neg=np.asarray(sentiment_neg)
    #     sentiment_pos=np.asarray(sentiment_pos)
    #     sentiment_neu=np.asarray(sentiment_neu)
    #     exc_mark=np.asarray(exc_mark)
    #     ques_mark=np.asarray(ques_mark)
    #     profane_words1=np.asarray(profane_words1)
    #     final=[padded_sequences,sentiment_subj.reshape(-1,1),sentiment_neg.reshape(-1,1),sentiment_pos.reshape(-1,1),sentiment_neu.reshape(-1,1),exc_mark.reshape(-1,1),ques_mark.reshape(-1,1),profane_words1.reshape(-1,1)]
    #     model=load_model('model.h5')
    #     pred=model.predict(final)
    #     final_predictions=[]
    #     if pred[0]>0.5:
    #         final_predictions.append(1)
    #     else:
    #         final_predictions.append(0)
            
    #     if 1 in final_predictions:
    #         return ("The above comment is sarcastic")
    #     else:
    #         return ("The above comment is non sarcastic")

In [20]:
y_pred=final_func1(data['comment'][129])

data['comment'][129]
print(y_pred)

[[0.57855624]]


In [21]:
data['label'][129]

0

In [22]:
def final_func2(text,y_orginal):
    final_predictions=[]
    y_pred=final_func1(text)
    print(y_pred)
    if y_pred[0]>0.5:
        final_predictions.append(1)
    else:
        final_predictions.append(0) 
    print(final_predictions)    
    if 1 in final_predictions:
        print("sarcastic")
    else:
        print("non_sarcastic")
        
    return final_predictions
        

In [23]:
final_func2('Looks like they choose locations far from the bridges to Gatineau, I wonder why.... ',1)

[[0.538448]]
[1]
sarcastic


[1]

In [24]:
print((data['comment'][55]))

I believe you can buy just the Happy Meal toys for $1.99 each
